In [1]:
from torchvision.transforms import transforms
transform=transforms.Compose([transforms.ToTensor(),transforms.Resize(size=(224, 224))])
from torchvision import datasets
BASE_PATH="/home/bubbles/Quantbit Internship/Quality Check/Tomato Freshness Dataset/train"
BASE_PATH_TEST="/home/bubbles/Quantbit Internship/Quality Check/Tomato Freshness Dataset/test"
train=datasets.ImageFolder(root=BASE_PATH,transform=transform)
test=datasets.ImageFolder(root=BASE_PATH_TEST,transform=transform)

In [2]:
from torch.utils.data import DataLoader
train_loader=DataLoader(train, batch_size=16,shuffle=True)
test_loader=DataLoader(test,batch_size=16,shuffle=False)
from torchvision import models
model=models.vgg16('IMAGENET1K_V1')

/home/bubbles/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [3]:
for param in model.features.parameters():
    param.requires_grad = False
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [4]:
from torch import nn
# model.fc=nn.Linear(1000,2)
# _modules['6']
model.classifier._modules['6']=nn.Linear(4096,2)
# model.classifier._modules['7']=nn.Sigmoid()
from torch.optim import Adam
clf=model.to('cpu')
opt=Adam(clf.parameters(),lr=1e-5)
loss_fn=nn.CrossEntropyLoss()

In [ ]:
import torch
for epoch in range(10):
  total_correct = 0
  total_samples = 0
  for batch in train_loader:
    X,y=batch
    X,y=X.to('cpu'),y.to('cpu')
    yhat=clf(X)
    # print(yhat,y.unsqueeze(1))
    # print(yhat.squeeze(1).float(),y)
    loss=loss_fn(yhat,y)

    opt.zero_grad()
    loss.backward()
    opt.step()
    # print(y,yhat)
    _, predicted = torch.max(yhat, 1)

        # Update the running total of correct predictions and samples
    total_correct += (predicted == y).sum().item()
    total_samples += y.size(0)

    # Calculate the accuracy for this epoch
  accuracy = 100 * total_correct / total_samples
  print(f'Epoch {epoch+1}: Accuracy = {accuracy:.2f}% Loss:{loss}')


/home/bubbles/.local/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1: Accuracy = 79.95% Loss:0.24163714051246643
Epoch 2: Accuracy = 93.25% Loss:0.008190997876226902
Epoch 3: Accuracy = 96.57% Loss:0.00865416880697012
Epoch 4: Accuracy = 97.20% Loss:0.017823010683059692
Epoch 5: Accuracy = 98.21% Loss:0.0036098994314670563


In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images=images.to('cpu')
        labels=labels.to('cpu')
        # calculate outputs by running images through the network
        outputs = clf(images.float())
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct // total} %')

In [1]:
model.save("Tomatofreshness.h5")

NameError: name 'model' is not defined

In [ ]:
from tensorflow.keras.models import load_model
model_json = model.to_json()
with open("model3.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
from keras.models import model_from_json

# load json and create model
json_file = open('model3.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model1 = model_from_json(loaded_model_json)
# load weights into new model
loaded_model1.load_weights("model.h5")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img,img_to_array
#model = load_model("../input/final-model-saved/fruit_n_veg_model.h5",compile=False)
import json
lab = training_set.class_indices
lab={k:v for v,k in lab.items()}
print(lab)
print(json.dumps(lab, indent=4))

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.applications import imagenet_utils
def output(location):
    img=load_img(location,target_size=(224,224,3))
    plt.imshow(img)
    img=img_to_array(img)
    
    img=img/255
    plt.imshow(img)
    img=np.expand_dims(img,[0])
    
    answer=model.predict(img)
  #  ans2 = imagenet_utils.decode_predictions(answer)
    print(answer)
    y_class = answer.argmax(axis=-1)
    #print(answer.argmax(axis=-))
    y = " ".join(str(x) for x in y_class)
    y = int(y)
    res = lab[y]
    return res

In [ ]:
import matplotlib.pyplot as plt
img="../input/fresh-and-stale-images-of-fruits-and-vegetables/fresh_apple/Screen Shot 2018-06-08 at 5.10.29 PM.png"
pic=image.load_img(img,target_size=(224,224,3))
#plt.imshow(pic)
output(img)

In [7]:
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [8]:
from PIL import Image

def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(size=(224, 224)),
    ])
    image = Image.open(image_path).convert('RGB')
    input_image = transform(image).unsqueeze(0)  # Add batch dimension
    return input_image

In [9]:
def predict_class(model, input_image):
    with torch.no_grad():
        output = model(input_image)
        _, predicted_class = torch.max(output, 1)
    return predicted_class.item()


In [18]:
user_image_path = input("Enter the path to your image: ")
user_input_image = preprocess_image(user_image_path)
predicted_class = predict_class(model, user_input_image)

#print(f"The predicted class for the given image is: {predicted_class}")
if predicted_class == 1:
    print("Tomato is classified as good.")
else:
    print("Tomato is classified as bad.")


Enter the path to your image:  /home/bubbles/Downloads/Tomato2.jpeg


Tomato is classified as good.
